In [2]:
using Fussy

using FileIO
using JLD2

using Plots
using Interact

using DataStructures
using StringCases

using LaTeXStrings

pgfplots()
return

In [3]:
cur_scans = load("../data/comparisons.jld2", "cur_scans")
cur_scans_demo = load("../data/comparisons_demo_tau.jld2", "cur_scans_demo")

cur_scans[:demo_steady_modified] = cur_scans_demo[:demo_steady]
cur_scans[:demo_pulsed_modified] = cur_scans_demo[:demo_pulsed]


Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.0, 1.33333, 1.66667, 2.0, 2.33333, 2.66667, 3.0  …  37.0, 37.3333, 37.6667, 38.0, 38.3333, 38.6667, 39.0, 39.3333, 39.6667, 40.0], :demo_pulsed, Fussy.Reactor[Fussy.Reactor
  T_bar: Float64 9.66667
  n_bar: Float64 10904.4
  I_P: Float64 2.4268
  R_0: Float64 0.0272552
  B_0: Float64 1185.43
  sigma_v: Float64 0.101257
  mode_scaling: Dict{Symbol,Float64}
  deck: Symbol demo_pulsed
  is_pulsed: Bool true
  is_symbolic: Bool false
  is_consistent: Bool false
  is_solved: Bool true
  is_valid: Bool false
  is_good: Bool true
  H: Float64 1.1
  Q: Float64 39.86
  epsilon: Float64 0.3226
  kappa_95: Float64 1.59
  delta_95: Float64 0.333
  nu_n: Float64 0.27
  nu_T: Float64 1.094
  Z_eff: Float64 2.584
  f_D: Float64 0.7753
  A: Float64 2.735
  l_i: Float64 1.155
  rho_m: Float64 0.0
  gamma: Float64 -2.02534
  N_G: Float64 1.2
  eta_T: Float64 0.3531
  eta_RF: Float64 0.4
  tau_FT: Float64 7273.0
  C_ejima: Float64 0.3
  pi: Float64 3.14

In [4]:
function filter_reactors!(reactor_list)
    filter!(Fussy.is_present, reactor_list)
    
    filter!(tmp_reactor -> tmp_reactor.is_valid, reactor_list)
    filter!(tmp_reactor -> tmp_reactor.is_good, reactor_list)
    
    filter!(tmp_reactor -> tmp_reactor.R_0 < 50, reactor_list)
    filter!(tmp_reactor -> tmp_reactor.B_0 < 50, reactor_list)

    filter!(tmp_reactor -> tmp_reactor.cost < 1, reactor_list)
    
    isempty(reactor_list) && return
    ( reactor_list[1].deck == :demo_steady ) && return
    ( reactor_list[1].deck == :act_1 ) && return
    
    filter!(tmp_reactor -> tmp_reactor.norm_P_E < 0.8, reactor_list)
end

return

In [5]:
function fix_lims!(cur_lims, x, y, xscale, yscale)
    cur_lims[1:2:3] /= 1.2
    cur_lims[2:2:4] *= 1.2
    
    if x == :cost
        cur_lims[1:2] = [0.001, 0.1]
    end
    
    if y == :cost
        cur_lims[3:4] = [0.001, 0.1]
    end
  
    if x == :W_M
        cur_lims[1:2] = [0.01, 100]
    end
    
    if y == :W_M
        cur_lims[3:4] = [0.01, 100]
    end
    
    if x == :B_0 || x == :R_0 || x == :h_CS
        cur_lims[1:2] = [0.05, 20]
    end
    
    if y == :B_0 || y == :R_0 || y == :h_CS
        cur_lims[3:4] = [0.05, 20]
    end
    
    if x == :T_bar || x == :I_P
        cur_lims[1:2] = [0.05, 40]
    end
    
    if y == :T_bar || y == :I_P
        cur_lims[3:4] = [0.05, 40]
    end
  
    if in(x, [:f_IN, :f_BS, :f_CD, :norm_P_W, :norm_beta_N, :norm_q_95])
         cur_lims[2] = 1.25
    end
    
    if in(y, [:f_IN, :f_BS, :f_CD, :norm_P_W, :norm_beta_N, :norm_q_95])
         cur_lims[4] = 1.25
    end
    
    if xscale == "log"
        iszero(cur_lims[1]) || plot!( xscale = :log )
    else
        cur_lims[1] = 0
    end
    
    if yscale == "log"
        iszero(cur_lims[3]) || plot!( yscale = :log )
    else
        cur_lims[3] = 0
    end
end

return

In [6]:
function make_comparisons(deck, x, y, xscale, yscale)  
    is_pulsed = deck == :proteus || startswith(string(deck), "demo")
    
    if is_pulsed
        other_label = startswith(string(deck), "demo") ? "modified" : "simple"
    else
        other_label = "steady"
    end
    
    other_type = Symbol("$(deck)_$(other_label)")
    
    min_x, max_x = Inf, -Inf
    min_y, max_y = Inf, -Inf
    
    scan_types = [deck, other_type]
    
    is_pulsed || reverse!(scan_types)
    
    for scan_type in scan_types
    
        cur_scan = deepcopy(cur_scans[scan_type])

        cur_dict = OrderedDict()

        cur_dict[:kink] = cur_scan.kink_reactors
        cur_dict[:beta] = cur_scan.beta_reactors
        cur_dict[:wall] = cur_scan.wall_reactors

        for (cur_key, cur_value) in cur_dict
            filter_reactors!(cur_value)

            isempty(cur_value) && continue

            min_branch_id = minimum(map(tmp_reactor -> tmp_reactor.branch_id, cur_value))
            max_branch_id = maximum(map(tmp_reactor -> tmp_reactor.branch_id, cur_value))

            for cur_branch_id in min_branch_id:max_branch_id
                tmp_value = filter(tmp_reactor -> tmp_reactor.branch_id == cur_branch_id, cur_value)
                ( length(tmp_value) > 1 ) || continue

                tmp_tt = map(tmp_reactor -> tmp_reactor.T_bar, tmp_value)

                tmp_xx = map(tmp_reactor -> getfield(tmp_reactor, x), tmp_value)
                tmp_yy = map(tmp_reactor -> getfield(tmp_reactor, y), tmp_value)

                Fussy.sort_lists!(tmp_tt, tmp_xx, tmp_yy)
                
                cur_label = string(cur_key)
                
                if scan_type == other_type 
                    cur_label = "$(other_label) - $(cur_label)"
                    cur_style = is_pulsed ? :dash : :solid
                else
                    cur_label = "pulsed - $(cur_label)"
                    cur_style = is_pulsed ? :solid : :dash
                end
                
                min_x = min(min_x, minimum(tmp_xx))
                min_y = min(min_y, minimum(tmp_yy))
                
                max_x = max(max_x, maximum(tmp_xx))
                max_y = max(max_y, maximum(tmp_yy))
                
                plot!(tmp_xx, tmp_yy, label=cur_label, style=cur_style)
            end
        end
        
    end
    
    cur_lims = [min_x, max_x, min_y, max_y]
    
    fix_lims!(cur_lims, x, y, xscale, yscale)
    
    if isdefined(Fussy, Symbol("$(deck)_solution"))
        cur_solution = getfield(Fussy, Symbol("$(deck)_solution"))()
        tmp_x = getfield(cur_solution, x)
        tmp_y = getfield(cur_solution, y)
        
        ( tmp_x == nothing ) && ( tmp_x = NaN )
        ( tmp_y == nothing ) && ( tmp_y = NaN )
        
        plot!(cur_lims[1:2], [tmp_y, tmp_y], color=:black, opacity=0.5, label="", style=:dot, width=2)
        plot!([tmp_x, tmp_x], cur_lims[3:4], color=:black, opacity=0.5, label="", style=:dot, width=2)
        scatter!([tmp_x], [tmp_y], color=:black, opacity=0.5, label="")
    end
    
    return cur_lims
end

return

In [6]:
cur_decks = [
    :proteus, # pulsed
    :charybdis, # steady state
    :arc, :act_1, :act_2,
    :demo_pulsed, :demo_steady
]

cur_plots = []

cur_x_list = [:B_0, :T_bar]
cur_y_list = [:R_0, :I_P]

for (cur_x, cur_y) in zip(cur_x_list, cur_y_list)
    for cur_deck in cur_decks
        cur_plot = plot(html_output_format=:svg)

        cur_lims = make_comparisons(cur_deck, cur_x, cur_y, :lin, :lin) 
        
        cur_xlabel = string(cur_x)
        cur_ylabel = string(cur_y)
         
        endswith(cur_xlabel, "_bar") && ( cur_xlabel = "\\overline $( cur_xlabel[1:end-4] )" )
        endswith(cur_ylabel, "_bar") && ( cur_ylabel = "\\overline $( cur_ylabel[1:end-4] )" )
        
        xlabel!(latexstring(cur_xlabel))
        ylabel!(latexstring(cur_ylabel))

        cur_title = string(cur_deck)
        cur_title = join(map(capitalize, split(cur_title, "_")), " ")
        title!(cur_title)

        xlims!(cur_lims[1:2]...)
        ylims!(cur_lims[3:4]...)

        plot!()

        savefig(cur_plot, "../images/comparisons/$(cur_deck)_$(cur_x)_vs_$(cur_y).tex")
    end
end

plot!()

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 Z M 3.765625 -3.3125 "/>
 
 
<path style="stroke:none;" d="M 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 L 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 Z M 4.171875 0 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 L 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 Z M 4.46875 -1.734375 "/>
 
 
<path style="stroke:none;" d="M 4.5625 -1.703125 C 4.5625 -2.515625 3.921875 -3.296875 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.765625 1.09375 -5.765625 C 1.390625 -6.265625 2.046875 -6.390625 2.40625 -6.390625 C 2.828125 -6.390625 3.375 -6.171875 3.375 -5.28125 C 3.375 -5.15625 3.34375 -4.578125 3.09375 -4.140625 C 2.796875 -3.65625 2.453125 -3.625 2.203125 -3.625 C 2.125 -3.609375 1.890625 -3.59375 1.8125 -3.59375 C 1.734375 -3.578125 1.671875 -3.5625 1.671875 -3.46875 C 1.671875 -3.359375 1.734375 -3.359375 1.90625 -3.359375 L 2.34375 -3.359375 C 3.15625 -3.359375 3.53125 -2.6875 3.53125 -1.703125 C 3.53125 -0.34375 2.84375 -0.0625 2.40625 -0.0625 C 1.96875 -0.0625 1.21875 -0.234375 0.875 -0.8125 C 1.21875 -0.765625 1.53125 -0.984375 1.53125 -1.359375 C 1.53125 -1.71875 1.265625 -1.921875 0.984375 -1.921875 C 0.734375 -1.921875 0.421875 -1.78125 0.421875 -1.34375 C 0.421875 -0.4375 1.34375 0.21875 2.4375 0.21875 C 3.65625 0.21875 4.5625 -0.6875 4.5625 -1.703125 Z M 4.5625 -1.703125 "/>
 
 
<path style="stroke:none;" d="M 4.6875 -1.640625 L 4.6875 -1.953125 L 3.703125 -1.953125 L 3.703125 -6.484375 C 3.703125 -6.6875 3.703125 -6.75 3.53125 -6.75 C 3.453125 -6.75 3.421875 -6.75 3.34375 -6.625 L 0.28125 -1.953125 L 0.28125 -1.640625 L 2.9375 -1.640625 L 2.9375 -0.78125 C 2.9375 -0.421875 2.90625 -0.3125 2.171875 -0.3125 L 1.96875 -0.3125 L 1.96

In [8]:
cur_best_dict = Dict()

Dict{Any,Any} with 0 entries

In [18]:
cur_decks = [
    :arc, :act_1, :act_2,
    :demo_steady, :demo_pulsed,
    :proteus, # pulsed
    :charybdis # steady state
]

cur_fields = fieldnames(Fussy.Reactor)
skipped_fields = [
    :rho_m, :gamma, :constraint, :branch_id, :speed_of_light, 
    :pi, :eta_RF, :eta_T, :C_ejima, :k, :wave_theta, :deck,
    :J_TF, :J_CS, :sigma_TF, :sigma_CS, :eta_LH
]

cur_reactors = nothing

solution_keys = []
input_keys = []

for cur_deck in cur_decks    
    cur_reactors = Dict(
        :base => getfield(Fussy, Symbol("$(cur_deck)_deck"))(),
    )
    
    if startswith(string(cur_deck), "demo_") || cur_deck == :proteus
        is_pulsed = true
        is_consistent = false
    else
        is_pulsed = false
        is_consistent = true
    end
    
    is_proto = cur_deck == :proteus || cur_deck == :charybdis
    
    if cur_deck == :proteus
        cur_reactors[:solution] = Fussy.minimize(Fussy.proteus_deck(is_pulsed=is_pulsed, is_consistent=is_consistent), :cost)
    elseif cur_deck == :charybdis
        cur_reactors[:solution] = Fussy.minimize(Fussy.charybdis_deck(is_pulsed=is_pulsed, is_consistent=is_consistent), :cost)
    else
        cur_reactors[:solution] = getfield(Fussy, Symbol("$(cur_deck)_solution"))()
    end
    
    cur_dict = Dict()
    
    for (cur_key, cur_reactor) in cur_reactors
        cur_dict[cur_key] = Dict()
        
        for cur_field in cur_fields
            startswith(string(cur_field), "max_") && continue
            in(cur_field, skipped_fields) && continue
            cur_value = getfield(cur_reactor, cur_field)

            isa(cur_value, Dict) && continue
            isa(cur_value, Vector) && continue
            Fussy.is_nothing(cur_value) && continue

            cur_type = eltype(cur_value)

            ( cur_type == SymEngine.Basic ) && continue 
            ( cur_type == Bool ) && continue 

            cur_dict[cur_key][cur_field] = cur_value
        end
    end
    
    if !is_proto
        tmp_solution_keys = setdiff(map(collect,map(keys,values(cur_dict)))...)
        tmp_input_keys = setdiff(keys(cur_dict[:base]), solution_keys)

        push!(tmp_solution_keys, :eta_CD)
        filter!(cur_key -> cur_key != :eta_CD, tmp_input_keys)

        if isempty(solution_keys) || isempty(input_keys)
            solution_keys = tmp_solution_keys
            input_keys = tmp_input_keys
        else
            @assert solution_keys == tmp_solution_keys
            @assert input_keys == tmp_input_keys
        end
    end
        
    if cur_deck != :proteus && cur_deck != :charybdis
        if haskey(cur_best_dict, cur_deck)
            best_reactor = cur_best_dict[cur_deck]
        else
            cur_suffix = ( startswith(string(cur_deck), "demo_") || cur_deck == :proteus ) ? "" : "_steady"

            cur_vector = vcat(
                cur_scans[Symbol("$(cur_deck)$(cur_suffix)")].kink_reactors,
                cur_scans[Symbol("$(cur_deck)$(cur_suffix)")].beta_reactors,
                cur_scans[Symbol("$(cur_deck)$(cur_suffix)")].wall_reactors
            )

            filter!(tmp_reactor -> tmp_reactor.is_valid, cur_vector)
            filter!(tmp_reactor -> tmp_reactor.is_good, cur_vector)

            cur_error_list = map(
                tmp_reactor -> 
                ( 
                    ( 1 - tmp_reactor.R_0/cur_reactors[:solution].R_0 ) ^ 2 + 
                    ( 1 - tmp_reactor.B_0/cur_reactors[:solution].B_0 ) ^ 2
                ),
                cur_vector
            )

            init_ordering = collect(1:length(cur_vector))

            Fussy.sort_lists!(cur_error_list, cur_vector, init_ordering)

            @assert abs(init_ordering[1]-init_ordering[2]) == 1
            @assert cur_vector[1].constraint == cur_vector[2].constraint

            cur_constraint = cur_vector[1].constraint

            cur_func = function(cur_T_bar)        
                tmp_reactor = Reactor(
                    cur_T_bar, deck=cur_deck, constraint=cur_constraint,
                    is_pulsed=is_pulsed, is_consistent=is_consistent
                )

                @assert tmp_reactor.is_good

                if is_consistent
                    cur_eta_CD = Fussy.converge(tmp_reactor)
                    @assert length(cur_eta_CD) == 1

                    tmp_reactor.eta_CD = cur_eta_CD[1]
                end

                Fussy.solve!(tmp_reactor)

                @assert tmp_reactor.is_valid
                @assert tmp_reactor.is_good

                cur_error = (
                    ( 1 - tmp_reactor.R_0/cur_reactors[:solution].R_0 ) ^ 2 + 
                    ( 1 - tmp_reactor.B_0/cur_reactors[:solution].B_0 ) ^ 2
                )

                cur_error
            end

            best_T_bar = Optim.minimizer(
                Optim.optimize(cur_func, sort([cur_vector[1].T_bar,cur_vector[2].T_bar])...)
            )

            best_reactor = Reactor(
                best_T_bar, deck=cur_deck, constraint=cur_constraint,
                is_pulsed=is_pulsed, is_consistent=is_consistent
            )

            if is_consistent
                cur_eta_CD = Fussy.converge(best_reactor)
                @assert length(cur_eta_CD) == 1

                best_reactor.eta_CD = cur_eta_CD[1]
            end

            Fussy.solve!(best_reactor)

            cur_best_dict[cur_deck] = best_reactor
        end
    end
    
    println()
    println("------")
    println(cur_deck)
    println("------")
    println()
    
    print(rpad("Input", 16))
    print(" | ")
    println(rpad("Value", 16))
    
    println("-"^16, " | ", "-"^16)
    
    for cur_key in input_keys
        tmp_key = string(cur_key)

        if endswith(tmp_key, "_bar")
            tmp_key = "\\overline $( tmp_key[1:end-4] )"
        end
        
        if contains(tmp_key, "_")
            first_part, last_part = split(tmp_key, "_")
            last_part = "\{$(last_part)\}"
            tmp_key = "$(first_part)_$(last_part)"
        end
        
        if any(cur_word -> startswith(tmp_key, "$(cur_word)_") || tmp_key == cur_word, ["eta", "tau", "kappa", "nu", "delta", "epsilon"])
             tmp_key = "\\$(tmp_key)"
        end
        
        print(rpad(latexstring(tmp_key), 16))
        print(" | ")
        println(rpad(getfield(cur_reactors[:base],cur_key), 16))  
    end
    
    println()
    
    print(rpad("Output", 16))
    print(" | ")
    print(rpad(is_proto ? "Value" : "Original", 16))
    
    if is_proto
        println("")
    else
        print(" | ")
        println(rpad("Fussy.jl", 16))
    end
    
    print("-"^16, " | ", "-"^16)
    
    if is_proto
        println("")
    else
        println(" | ", "-"^16)
    end
    
    for cur_key in vcat(solution_keys, [:W_M, :cost, :sigma_v])
        tmp_key = string(cur_key)
        
        if endswith(tmp_key, "_bar")
            tmp_key = "\\overline $( tmp_key[1:end-4] )"
        end
        
        if contains(tmp_key, "_")
            first_part, last_part = split(tmp_key, "_")
            last_part = "\{$(last_part)\}"
            tmp_key = "$(first_part)_$(last_part)"
        end
        
        if any(cur_word -> startswith(tmp_key, "$(cur_word)_") || tmp_key == cur_word, ["eta", "tau", "kappa", "nu", "delta", "epsilon"])
             tmp_key = "\\$(tmp_key)"
        end
        
        print(rpad(latexstring(tmp_key), 16))
        print(" | ")
        try
            print(rpad(signif(round(getfield(cur_reactors[:solution],cur_key),4),4), 16))  
        catch
            print(rpad("-",16))
        end
        
        if is_proto
            println("")
        else
            print(" | ")
            tmp_value = signif(round(getfield(best_reactor,cur_key),4),4)

            isapprox(tmp_value, 0.0, atol=5e-5) && ( tmp_value = 0.0 )
            println(rpad(tmp_value, 16))  
        end
    end
    
end


------
arc
------

Input            | Value           
---------------- | ----------------
$\tau_{FT}$      | 1.6e9           
$f_{D}$          | 0.9             
$\kappa_{95}$    | 1.84            
$\epsilon$       | 0.3333          
$\nu_{n}$        | 0.385           
$A$              | 2.5             
$Q$              | 13.6            
$\delta_{95}$    | 0.333           
$l_{i}$          | 0.67            
$H$              | 1.8             
$\nu_{T}$        | 0.929           
$Z_{eff}$        | 1.2             
$B_{CS}$         | 12.77           
$N_{G}$          | 0.67            

Output           | Original         | Fussy.jl        
---------------- | ---------------- | ----------------
$f_{CD}$         | 0.37             | 0.4367          
$q_{95}$         | 7.2              | 6.177           
$beta_{N}$       | 0.0259           | 0.0259          
$R_{0}$          | 3.3              | 3.587           
$\overline n$    | 1.3              | 1.131           
$f_{BS}$         |

------
proteus
------

Input            | Value           
---------------- | ----------------
$\tau_{FT}$      | 7200.0          
$f_{D}$          | 0.9             
$\kappa_{95}$    | 1.8             
$\epsilon$       | 0.3             
$\nu_{n}$        | 0.4             
$A$              | 2.5             
$Q$              | 25.0            
$\delta_{95}$    | 0.35            
$l_{i}$          | 0.632845        
$H$              | 1.0             
$\nu_{T}$        | 1.1             
$Z_{eff}$        | 1.75            
$B_{CS}$         | 20.0            
$N_{G}$          | 0.9             

Output           | Value           
---------------- | ----------------
$f_{CD}$         | 0.0             
$q_{95}$         | 2.5             
$beta_{N}$       | 0.028           
$R_{0}$          | 6.11            
$\overline n$    | 1.158           
$f_{BS}$         | 0.2675          
$volume$         | 732.6           
$\overline T$    | 11.25           
$I_{P}$          | 15.54           
$P_{

In [19]:
# ss
0.7232 * 8.978

6.4928896

In [20]:
# pu
0.2675 * 15.54

4.15695